In [1]:
from deepCNN import deepCNN
from utils import *
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [2]:
model = deepCNN()
print(model)

deepCNN(
  (ConvLayer1): Sequential(
    (0): Conv2d(1, 25, kernel_size=(1, 10), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): Conv2d(25, 25, kernel_size=(22, 1), stride=(1, 1))
    (3): ELU(alpha=1.0)
    (4): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  )
  (elu): ELU(alpha=1.0)
  (maxpool): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(25, 50, kernel_size=(1, 10), stride=(1, 1))
  (conv3): Conv2d(50, 100, kernel_size=(1, 10), stride=(1, 1))
  (conv4): Conv2d(100, 200, kernel_size=(1, 10), stride=(1, 1))
)


In [3]:
X_train, y_train, X_test, y_test = init_data(subject=None, verbose=True)

train_loader, val_loader = load_data(X_train, y_train, X_test, y_test, verbose=True)



print(train_loader)

In [4]:
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 10  # Number of epochs to train for

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

RuntimeError: Given groups=1, weight of size [25, 1, 1, 10], expected input[1, 32, 22, 1000] to have 1 channels, but got 32 channels instead